In [7]:
import pandas as pd

import requests as rq

from bs4 import BeautifulSoup as bs

from datetime import datetime
from tqdm.notebook import tqdm
import time

In [34]:
# Функция parse_auto_links() возвращает ссылки на авто.
#
# -price_range(type list) задает диапозон цен по которым ищем авто.
#
# -num_of_pages(type int) задает количество страниц, среди которых ищем ссылки на авто.

def parse_auto_links(price_range, num_of_pages=99):
    
    # Время запуска функции
    begin_time = datetime.now()
    
    # Создаем список для нашых ссылок на авто.
    all_auto_links = []
    
    # Перебираем в цикле цены. Цена задается "от", 
    # т.е. "от 100_000 и выше".
    for price in price_range:
        
        # Создаем список, в который сложем все страницы с авто.
        all_auto_pages = []     
        
        print('Searching in price range > '+ str(price))
        
        # Заполняем созданный список страницами с авто.
        for page_number in range(1,num_of_pages+1):
            all_auto_pages.append('https://auto.ru/moskva/cars/bmw/used/?output_type=list&page={a}&price_from={b}&sort=price-asc'.format(a=page_number, b=price))
        
        # Перебираем в цилке список страниц с авто.
        # Делаем запрос страницу. После этого вытаскиваем с нее все ссылки на авто.
        for url in tqdm(all_auto_pages):
            
            response = rq.get(url)
            page = bs(response.text, 'html.parser')
            search_links = page.find_all('a', class_='Link ListingItemTitle-module__link')
            
            for auto_link in search_links:
                all_auto_links.append(auto_link.get('href'))
        
        print('\n')
    print('Found {} links'.format(len(all_auto_links)))
    print('Done')
    
    # Выводим время выполнения функции
    print('Execution time: {}'.format(datetime.now()-begin_time))
    
    return all_auto_links

In [35]:
price_range = [1, 1_100_000]

auto_links = parse_auto_links(price_range)

Searching in price range > 1





Searching in price range > 1100000





Finded 6997 links
Done
Execution time: 0:12:34.042749


In [122]:
time = datetime.now()

file=open('bmw_auto_links.txt','w')
file.write('Time update: ' + str(time) + '\n')
for link in auto_links:
    file.write(link+'\n')

file.close()

In [91]:
# Функция parse_auto_data() собирает информацию об авто.
#
# -auto_links(type list) список ссылок на авто, полученый с помошью функции parse_auto_links

def parse_auto_data(auto_links):
    
    # Время запуска функции
    begin_time = datetime.now()
    
    # Создаем список словарей с данными об авто
    auto_data = []
    
    # Создаем словарь, в котором key - car_index
    #                            value - словарь с данными об авто
    car_index = 0
    all_auto = {}
    
    # Перебираем в цикле все ссылки на авто
    for url in tqdm(auto_links):
        
        auto_info = []
        car_index += 1
        
        #Создаем словарь с данными об авто
        auto = {}
        
        rp = rq.get(url)
        rp.encoding = 'utf-8'
        page = bs(rp.text, 'html.parser')
        info = page.find_all('span', class_='CardInfo__cell')
        
        # Собираем через цикл всю информацию об авто
        for data in info:
            auto_info.append(data.text)
            
        # Преобразуем список с информацией об авто в словарь, где нечетные индекс это key,
        # а четный индекс value
        for i in range(0,len(auto_info)-5,2):
            auto[auto_info[i]] = auto_info[i+1]
        
        all_auto[car_index] = auto
        
    auto_data.append(all_auto)
    
    print('Done')
    
    # Выводим время выполнения функции
    print('Execution time: {}'.format(datetime.now()-begin_time))
    
    return auto_data

In [94]:
auto_dataset = parse_auto_data(auto_links[0:2])


Done
Execution time: 0:00:03.317859


In [123]:
auto_dataset

[{1: {'год выпуска': '1986',
   'Пробег': '220\xa0000\xa0км',
   'Кузов': 'седан',
   'Цвет': 'золотистый',
   'Двигатель': '2.7 л / 125\xa0л.с. / Бензин',
   'Налог': '3\xa0125\xa0₽ / год',
   'Коробка': 'механическая',
   'Привод': 'задний',
   'Руль': 'Левый',
   'Состояние': 'Не требует ремонта',
   'Владельцы': '2\xa0владельца',
   'ПТС': 'Оригинал',
   'Таможня': 'Растаможен'},
  2: {'год выпуска': '1982',
   'Пробег': '250\xa0000\xa0км',
   'Кузов': 'седан',
   'Цвет': 'белый',
   'Двигатель': '2.0 л / 125\xa0л.с. / Бензин',
   'Налог': '3\xa0125\xa0₽ / год',
   'Коробка': 'механическая',
   'Привод': 'задний',
   'Руль': 'Левый',
   'Состояние': 'Не требует ремонта',
   'Владельцы': '1\xa0владелец'}}]